# PoC: WeCookio multiagent CrewAI INC

This notebook demonstrates the WeCookio CrewAI implementation with cost-optimized model selection using uv package manager.

### Process.Sequential without delegation

### Installing Required Packages with uv

In [ ]:
# Install required packages using uv
!uv add ipykernel crewai crewai-tools boto3 botocore pyyaml ipywidgets

### Import Libraries and Load Configuration

In [ ]:
import yaml
from crewai import Crew, Agent, Task, Process
import boto3
import json

print("Libraries imported successfully!")

### Loading AWS Enviroment variables

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
AWS_ACCESS_KEY_ID=os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY=os.getenv("AWS_SECRET_ACCESS_KEY")
AWS_DEFAULT_REGION=os.getenv("AWS_REGION")

### Load Configuration Files

In [ ]:
#Check base folder for relative paths
import os
print (os.getcwd())
prefix = "crew/latest/"
# Load the integration configuration
with open(prefix+'config/agent-sequential/integration_config.yaml', 'r') as file:
    integration_config = yaml.safe_load(file)
# Load the crew configuration
with open(prefix+'config/agent-sequential/crew_config.yaml', 'r') as file:
    crew_config = yaml.safe_load(file)
with open(prefix+'config/agent-sequential/agents.yaml', 'r') as file:
    agents_config = yaml.safe_load(file)
with open(prefix+'config/agent-sequential/tasks.yaml', 'r') as file:
    tasks_config = yaml.safe_load(file)

print("Configuration files loaded successfully!")
print(f"Integration config keys: {list(integration_config.keys())}")
print(f"Crew Orchestration keys: {list(crew_config.keys())}")
print(f"Agents config keys: {list(agents_config.keys())}")
print(f"Tasks config keys: {list(tasks_config.keys())}")

###  Model Selection based on complexity

In [ ]:
def select_model_for_complexity(complexity, integration_config):
    """Select the appropriate model based on task complexity"""
    try:
        complexity = complexity + "_model"
        return integration_config["aws_bedrock"][complexity]
    except KeyError:
        raise ValueError(f"Invalid complexity level: {complexity}")

# Example usage
high_complexity_model = select_model_for_complexity("high_complexity", integration_config)
medium_complexity_model = select_model_for_complexity("medium_complexity", integration_config)
low_complexity_model = select_model_for_complexity("low_complexity", integration_config)
ultra_low_complexity_model = select_model_for_complexity("ultra_low_complexity", integration_config)

print(f"High complexity model: {high_complexity_model['model_id']}")
print(f"Medium complexity model: {medium_complexity_model['model_id']}")
print(f"Low complexity model: {low_complexity_model['model_id']}")
print(f"Ultra-Low complexity model: {ultra_low_complexity_model['model_id']}")

### Create Agents with Model-Specific Configuration

In [ ]:
from crewai import LLM
from pprint import pprint

def create_agent_with_model(agent_config, model_config):
    """Create an agent with specific model configuration"""
    llm = LLM(
        model=model_config["model_id"],
        region_name=AWS_DEFAULT_REGION,
        max_tokens=model_config.get("model_kwargs").get("max_tokens", 4000),
        temperature=model_config.get("model_kwargs").get("temperature", 0.6),
        top_p=model_config.get("model_kwargs").get("top_p", 0.9),
        api_key=AWS_SECRET_ACCESS_KEY
    )
    return Agent(
        role=agent_config["role"],
        goal=agent_config["goal"],
        backstory=agent_config["backstory"],
        verbose=agent_config.get("verbose", True),
        allow_delegation=agent_config.get("allow_delegation", False),
        prompt_template=agent_config.get("prompt", None),
        reasoning=agent_config.get("reasoning", False),
        max_reasoning_attempts=agent_config.get("max_reasoning_attempts", 3),
        #response_template="{{ .Response }}",
        max_iterations=model_config.get("model_kwargs").get("max_iterations", 3),
        max_retry_limit=model_config.get("model_kwargs").get("max_retry_limit", 2),
        max_execution_time=model_config.get("model_kwargs").get("max_execution_time", 30),
        llm=llm
    )

def print_agent_info(agent, model_config) -> None:
    """Print agent creation information."""
    print(
        f"Created agent: {agent.id}\n"
        f"Agent Role: {agent.role}\n"
        f"Using model: {model_config.get('model_id')}"
    )
    print("\n")

In [ ]:

#agent_name = "Culinary Coordinator"
agent_config = agents_config["Culinary Coordinator"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_coordinator_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_coordinator_agent, model_config)

#agent_name = "Dietary Compliance Specialist"
agent_config = agents_config["Dietary Compliance Specialist"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
dietary_compliance_specialist_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(dietary_compliance_specialist_agent, model_config)

agent_name = "Ingredient Substitution Expert"
agent_config = agents_config["Ingredient Substitution Expert"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
ingredient_substitution_expert_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(ingredient_substitution_expert_agent, model_config)

agent_name = "Culinary Experience Optimizer"
agent_config = agents_config["Culinary Experience Optimizer"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
culinary_experience_optimizer_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(culinary_experience_optimizer_agent, model_config)

agent_name = "Quality Assurance Chef"
agent_config = agents_config["Quality Assurance Chef"]
agent_type=agent_config["name"]
model_config = select_model_for_complexity(crew_config["model_assignment"]["agent_model_mapping"][agent_type], integration_config)
quality_assurance_chef_agent = create_agent_with_model(agent_config, model_config)
print_agent_info(quality_assurance_chef_agent, model_config)





### Create Tasks with Complexity-Based Model Selection

In [ ]:
def create_task_with_agent(task_config, agent):
    """Create a task with the appropriate agent based on complexity"""
    return Task(
        description=task_config["description"],
        expected_output=task_config["expected_output"],
        agent=agent,
        async_execution=task_config.get("async_execution", False)
    )

# Example: Create a task for the coordinator
#task_name="Initial Recipe Analysis"
#inital_recipe_config = tasks_config[task_name]
#pprint.pprint(inital_recipe_config)
#print(f"Description type: {type(inital_recipe_config.get("description", "Not Found"))}")
#print(f"Expected output type: {type(inital_recipe_config.get("expected_output", "Not Found"))}")
#print(f"Template type: {type(inital_recipe_config.get("Template", "Not Found"))}")
#print(f"Async execution type: {type(inital_recipe_config.get("async_execution", "Not Found"))}")

def print_task_info(task) -> None:
    """Print information about a created task."""
    print(f"Created task: {task.id}")
    print(f"Task description: {task.description}")
    print(f"Assigned to agent: {task.agent.role}")
    print("\n")

In [ ]:
# task_name = "Initial Recipe Analysis"
task_config = tasks_config["Initial Recipe Analysis"]
inital_recipe_analysis_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {inital_recipe_analysis_task.id}")
print(f"Task description: {inital_recipe_analysis_task.description}")
print(f"Assigned to agent: {inital_recipe_analysis_task.agent.role}")
print("\n")

# task_name = "Dietary Compliance Validation"
task_config = tasks_config["Dietary Compliance Validation"]
dietary_compliance_validation_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {dietary_compliance_validation_task.id}")
print(f"Task description: {dietary_compliance_validation_task.description}")
print(f"Assigned to agent: {dietary_compliance_validation_task.agent.role}")
print("\n")

task_name = "Ingredient Substitution Strategy"
task_config = tasks_config["Ingredient Substitution Strategy"]
ingredient_substitution_strategy_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {ingredient_substitution_strategy_task.id}")
print(f"Task description: {ingredient_substitution_strategy_task.description}")
print(f"Assigned to agent: {ingredient_substitution_strategy_task.agent.role}")
print("\n")

# task_name = "Culinary Experience Enhancement"
task_config = tasks_config["Culinary Experience Enhancement"]
culinary_experience_enhancement_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {culinary_experience_enhancement_task.id}")
print(f"Task description: {culinary_experience_enhancement_task.description}")
print(f"Assigned to agent: {culinary_experience_enhancement_task.agent.role}")
print("\n")

# task_name = "Final Quality Assurance"
task_config = tasks_config["Final Quality Assurance"]
final_quality_assurance_task = create_task_with_agent(task_config, eval(task_config["agent"]))

print(f"Created task: {final_quality_assurance_task.id}")
print(f"Task description: {final_quality_assurance_task.description}")
print(f"Assigned to agent: {final_quality_assurance_task.agent.role}")
print("\n")

### Create Crew 

In [ ]:
# Create a simple crew for demonstration
embedder={
        "provider": "bedrock",
        "config": {
            "model": "amazon.titan-embed-text-v2:0"
        }
    }

crew = Crew(
    agents=[culinary_coordinator_agent,  ingredient_substitution_expert_agent, culinary_experience_optimizer_agent, quality_assurance_chef_agent],
    tasks=[inital_recipe_analysis_task, ingredient_substitution_strategy_task, culinary_experience_enhancement_task, final_quality_assurance_task],
    verbose=True,
    memory=False, 
    #memory=True,
    #embedder=embedder,
    process=eval(crew_config.get("execution").get("process")),
    cache=crew_config.get("cache", True),
    #planning=True,
    #planner=select_model_for_complexity("low_complexity", integration_config)
)

print("Crew created successfully!")
print("Flow model: ", crew_config.get("execution").get("process"))
print(f"Number of agents: {len(crew.agents)}")
print(f"Number of tasks: {len(crew.tasks)}")
print(f"Memory enabled: {crew.memory}")
print(f"Cache enabled: {crew.cache}")

### Recipe Parameters

In [ ]:
# Example recipe for analysis
recipe_request = {
    "food_name": "Canelones de ternera XXL al Pedro Ximenez",
    "servings": 4,
    "intolerances": [
        "Milk",
        "Eggs",
        "Beef",
        "Corn",
        "Gluten",
        "Rice",
        "Garlic"
    ],
    "exclusions": ["Pepper"],
    "preferences": [],
    "output_language": "es",
    "country": "ES"
}

print("Sample request for analysis:")
print(json.dumps(recipe_request, indent=2))

### Run the Crew (Requires AWS Credentials)

In [ ]:
# Uncomment the following lines if you have AWS credentials configured
# and want to actually run the crew

try:
    result = crew.kickoff(inputs=recipe_request)
    print("\nCrew execution result:")
    print(result)
except Exception as e:
    print(f"Error running crew: {e}")
    print("Make sure you have AWS credentials configured and the required permissions.")

#print("Notebook execution completed successfully!")
#print("\nTo run the actual crew:")
#print("1. Configure AWS credentials")
#print("2. Ensure Bedrock access permissions")
#print("3. Uncomment the execution code above")